In this second part of the lab, we will implement a language identifier trained on the same data, but using Logistic Regression instead of Naive Bayes.

In [1]:
import io, sys, math
import numpy as np
from collections import defaultdict

This function is used to build the dictionary, or vocabulary, which is a mapping from strings (or words) to integers (or indices). This will allow to build vector representations of documents. 

In [2]:
def build_dict(filename, threshold=1):
    fin = io.open(filename, 'r', encoding='utf-8')
    word_dict, label_dict = {}, {}
    counts = defaultdict(lambda: 0)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        if not label in label_dict:
            label_dict[label] = len(label_dict)

        for w in tokens[1:]:
            counts[w] += 1
            
    for k, v in counts.items():
        if v > threshold:
            word_dict[k] = len(word_dict)
    return word_dict, label_dict

In [3]:
word_dict, label_dict = build_dict("/content/drive/MyDrive/NLP_Week_1_Labs_2022/session1/train1.txt")

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This function is used to load the training dataset, and build vector representations of the training examples. In particular, a document or sentence is represented as a bag of words. Each example correspond to a sparse vector ` x` of dimension `V`, where `V` is the size of the vocabulary. The element `j` of the vector `x` is the number of times the word `j` appears in the document.

In [4]:
def load_data(filename, word_dict, label_dict):
    fin = io.open(filename, 'r', encoding='utf-8')
    data = []
    dim = len(word_dict)
    for line in fin:
        tokens = line.split()
        label = tokens[0]

        yi = label_dict[label]
        xi = np.zeros(dim)
        for word in tokens[1:]:
            if word in word_dict:
                wid = word_dict[word]
                xi[wid] += 1.0
        data.append((yi, xi))
    return data

First, let's implement the softmax function. Don't forget numerical stability!

In [5]:
def softmax(x):
  ### FILL CODE
  ex = np.exp(x - np.max(x))
  return ex/np.sum(ex, axis =0)

Now, let's implement the main training loop, by using stochastic gradient descent. The function will iterate over the examples of the training set. For each example, we will first compute the loss, before computing the gradient and performing the update.

In [11]:
def sgd(w, data, niter):
    nlabels, dim = w.shape
    for iter in range(niter):
      loss = 0.0
      for y,x in data:
        z = np.dot(w,x)
        # label predict
        lab_pred = softmax(z)

        # compute loss softmax
        loss += -(1/len(data))*np.sum(np.log(lab_pred[y]))
        
        # compute gradient softmax
        grad = lab_pred - y
        zero_vec=np.zeros(nlabels)
        zero_vec[y]= 1.0
        grad= grad-zero_vec
        grad = grad.reshape(nlabels, 1)*x.reshape((1, dim))

        # Update w
        w = w - 0.01*grad
      print("iteration: %02d loss: %.3f" % (iter, loss))
    return w

The next function will predict the most probable label corresponding to example `x`, given the trained classifier `w`.

In [12]:
def predict(w, x):
  ## FILL CODE
  z = w@x
  lab_pred = softmax(z)
  pred = np.argmax(lab_pred)
  return pred


Finally, this function will compute the accuracy of a trained classifier `w` on a validation set.

In [35]:
def compute_accuracy(w, valid_data):
  acc = 0.00
  for y, x in valid_data:
    
    lab_pred = predict(w, x)
    y_equal= np.where(lab_pred==y,1,0)
    if y_equal == 1:
      acc +=1
  acc = acc/len(valid_data)
  return acc  
    ## FILL CODE

In [36]:
print("")
print("** Logistic Regression **", )
print("")

word_dict, label_dict = build_dict("/content/drive/MyDrive/NLP_Week_1_Labs_2022/session1/train1.txt")
train_data = load_data("/content/drive/MyDrive/NLP_Week_1_Labs_2022/session1/train1.txt", word_dict, label_dict)
valid_data = load_data("/content/drive/MyDrive/NLP_Week_1_Labs_2022/session1/valid1.txt", word_dict, label_dict)

nlabels = len(label_dict)
dim = len(word_dict)
w = np.zeros([nlabels, dim])
w = sgd(w, train_data, 5)
print("")
print("Validation accuracy: %.3f" % compute_accuracy(w, valid_data))
print("")


** Logistic Regression **

iteration: 00 loss: 1.643
iteration: 01 loss: 1.133
iteration: 02 loss: 0.931
iteration: 03 loss: 0.812
iteration: 04 loss: 0.730

Validation accuracy: 0.889

